# Pull Alpaca API data for ARKK holdings

In [1]:
import os
import requests
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from datetime import datetime, timedelta
import questionary

In [2]:
load_dotenv()

True

In [3]:
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

In [4]:
alpaca = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version="v2")

In [5]:
print(alpaca)

## ARK ETF - Alpaca Data Pull

In [6]:
ARK_ETF = ["TSLA", "ROKU", "TDOC", "MSTR", "SQ", "SHOP", "TTWO", "TWLO", "SPOT", "NTLA", "CRSP", "EXAS", "Z", "TWTR", "TER","DOCU","CGEN"]

In [7]:
timeframe = "1D"

In [8]:
start_date = pd.Timestamp("2018-09-01", tz="America/New_York").isoformat()
end_date = pd.Timestamp("2021-08-01", tz="America/New_York").isoformat()

In [9]:
 ## This code artificially creates 3 years -- could replace code in line 8 --

In [10]:
# today = datetime.now()
# startdate = today - timedelta(days=3*365)

In [11]:
# this is to convert the datetime format to the pd.Timestamp that alpaca api syntax requirement

In [12]:
# end_date = pd.Timestamp(today.strftime("%Y-%m-%d"), tz="America/New_York").isoformat()
# start_date = pd.Timestamp(startdate.strftime("%Y-%m-%d"), tz="America/New_York").isoformat()

In [13]:
ARK_ETF_df = alpaca.get_barset(
    ARK_ETF,
    timeframe,
    start = start_date,
    end = end_date,
    limit=1000
).df

In [14]:
ARK_ETF_df.head()

CGEN                             CRSP         \
                           open  high   low close  volume   open   high   
time                                                                      
2018-09-04 00:00:00-04:00  3.90  4.00  3.75  3.90  117029  58.50  59.00   
2018-09-05 00:00:00-04:00  3.80  3.95  3.80  3.95  115535  55.50  55.50   
2018-09-06 00:00:00-04:00  3.90  3.95  3.65  3.95  200494  53.17  53.72   
2018-09-07 00:00:00-04:00  3.95  3.95  3.75  3.85  176952  50.00  51.73   
2018-09-10 00:00:00-04:00  3.95  4.00  3.85  3.95  159873  50.66  50.85   

                                                   ...   TWTR          \
                              low  close   volume  ...   open    high   
time                                               ...                  
2018-09-04 00:00:00-04:00  53.600  55.50  1032358  ...  34.75  35.125   
2018-09-05 00:00:00-04:00  52.005  53.44   606929  ...  34.65  34.700   
2018-09-06 00:00:00-04:00  49.580  49.84   558842  ...  32.86  32.950   
2018-09-07 00:00:00-04:00  49.360  50.12   431793  ...  30.31  31.390   
2018-09-10 00:00:00-04:00  47.500  49.67  1106214  ...  30.50  30.600   

                                                         Z                 \
                               low  close    volume   open    high    low   
time                                                                        
2018-09-04 00:00:00-04:00  34.4800  34.84  12200652  48.55  48.900  47.56   
2018-09-05 00:00:00-04:00  32.5100  32.73  32198973  47.82  48.270  46.91   
2018-09-06 00:00:00-04:00  30.6200  30.82  33189835  47.68  47.980  43.87   
2018-09-07 00:00:00-04:00  29.8200  30.49  28893185  44.58  46.115  44.52   
2018-09-10 00:00:00-04:00  29.9531  30.54  15451664  45.00  46.200  44.60   

                                           
                           close   volume  
time                                       
2018-09-04 00:00:00-04:00  47.72  1177704  
2018-09-05 00:00:00-04:00  47.67  1171127  
2018-09-06 00:00:00-04:00  44.62  2537771  
2018-09-07 00:00:00-04:00  44.93  1830479  
2018-09-10 00:00:00-04:00  46.12  1103775  

[5 rows x 85 columns]

In [15]:
## saving ARK_ETF_portfolio to csv

In [16]:
# ARK_ETF_df.to_csv("Ark_ETF.csv")

### Competitors - Alpaca Data Pull

In [17]:
competitors_list = ["NIO", "NFLX", "PFE", "MSFT", "V", "PYPL", "CRM", "ATVI", "VG", "TTD", "IONS", "NVAX", "AMGN", "RDFN", "SNAP"]

In [18]:
timeframe = "1D"

In [19]:
start_date = pd.Timestamp("2018-09-01", tz="America/New_York").isoformat()
end_date = pd.Timestamp("2021-08-01", tz="America/New_York").isoformat()

In [20]:
 ## This code artificially creates 3 years -- could replace code in line 16 --

In [21]:
today = datetime.now()
startdate = today - timedelta(days=3*365)

In [22]:
end_date = pd.Timestamp(today.strftime("%Y-%m-%d"), tz="America/New_York").isoformat()
start_date = pd.Timestamp(startdate.strftime("%Y-%m-%d"), tz="America/New_York").isoformat()

In [23]:
competitors_df = alpaca.get_barset(
    competitors_list,
    timeframe,
    start = start_date,
    end = end_date,
    limit=1000
).df

In [24]:
competitors_df.head()

AMGN                                      ATVI  \
                             open    high       low   close   volume   open   
time                                                                          
2018-08-16 00:00:00-04:00  194.46  197.13  192.7300  196.48  2041279  69.68   
2018-08-17 00:00:00-04:00  196.24  198.51  195.2800  197.42  1750815  69.51   
2018-08-20 00:00:00-04:00  197.89  198.50  196.8700  197.54  1200772  69.21   
2018-08-21 00:00:00-04:00  197.93  198.40  196.7300  197.32  1212006  69.27   
2018-08-22 00:00:00-04:00  197.15  198.75  195.8337  197.86  1452849  69.58   

                                                           ...       V  \
                            high      low  close   volume  ...    open   
time                                                       ...           
2018-08-16 00:00:00-04:00  69.99  68.9100  69.69  8359894  ...  140.75   
2018-08-17 00:00:00-04:00  69.67  68.3300  69.14  4825534  ...  140.78   
2018-08-20 00:00:00-04:00  69.39  68.2200  68.94  3761930  ...  141.54   
2018-08-21 00:00:00-04:00  70.26  69.0400  69.67  4748136  ...  141.00   
2018-08-22 00:00:00-04:00  71.52  69.5001  71.43  7029012  ...  139.85   

                                                                  VG           \
                               high      low   close   volume   open     high   
time                                                                            
2018-08-16 00:00:00-04:00  141.4900  140.420  140.64  3688784  14.13  14.4600   
2018-08-17 00:00:00-04:00  141.7100  140.565  141.38  3938667  14.23  14.5900   
2018-08-20 00:00:00-04:00  141.6000  140.700  140.94  2815641  14.57  14.6115   
2018-08-21 00:00:00-04:00  141.4574  139.800  140.02  3455094  14.53  14.5500   
2018-08-22 00:00:00-04:00  141.5300  139.850  141.15  2981091  14.27  14.4100   

                                                     
                             low   close     volume  
time                                                 
2018-08-16 00:00:00-04:00  14.08  14.240   899309.0  
2018-08-17 00:00:00-04:00  14.14  14.560  1050613.0  
2018-08-20 00:00:00-04:00  14.30  14.485   598652.0  
2018-08-21 00:00:00-04:00  14.30  14.300   942216.0  
2018-08-22 00:00:00-04:00  14.14  14.340   743927.0  

[5 rows x 75 columns]

In [25]:
## saving competitors_portfolio to csv

In [26]:
# competitors_list.to_csv("competitors_list.csv")

In [27]:
## Dictionary & weights 

In [28]:
ARK_weight = {
    "TSLA": 0.17,
    "ROKU": 0.08,
    "TDOC": 0.057,
    "MSTR": 0.11,
    "SQ": 0.057,
    "SHOP": 0.043,
    "TTWO": 0.04,
    "TWLO": 0.0368,
    "SPOT": 0.0535,
    "NTLA": 0.0301,
    "CRSP": 0.0526,
    "EXAS": 0.048,
    "Z": 0.052,
    "TWTR": 0.043,
    "TER": 0.02,
    "DOCU": 0.082,
    "CGEN": 0.025
}

In [29]:
competitors_weight = {}

In [30]:
for ticker in ARK_weight:
    competitors_weight [competitors_df [ARK_ETF.index(ticker)]] = ARK_weight[ticker]                

KeyError: 0

In [ ]:
print(competitors_weight)